# Facebook爬蟲

# 定義函數供後續使用

在這裡我們先定義一個函數，希望把網頁中各篇貼文的連結都找出來!
ulr放我們要爬的Facebook網址，n是稍後要送出幾次滾動網頁到底部的命令，藉以加載更多資料。

In [2]:
def FindLinks(url, n):
    Links = []
    driver.get(url)
    for i in range(n):
        time.sleep(1)
        driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
    # 這裡會跳出要我們登入的大畫面，找到「稍後再說」的按鈕並點擊
    driver.find_element_by_xpath('//a[@id="expanding_cta_close_button"]').click()
    soup = BeautifulSoup(driver.page_source)
    posts = soup.findAll('div', {'class':'clearfix y_c3pyo2ta3'})
    for i in posts:
        Links.append('https://www.facebook.com' + i.find('a',{'class':'_5pcq'}).attrs['href'].split('?',2)[0])
    return Links

定義一個展開所有留言的函數，透過while迴圈反覆搜尋與點擊「看更多留言」、「看更多回覆」與「看完整貼文內容」等按鈕。
在過程中會出現請我們登入或註冊的彈跳視窗，但我們不確定到底什麼時候會跳出，因此需要在過程中反覆偵測是有出現這個彈窗，若有就點擊「Not Now」

In [3]:
# 展開url中的所有留言 與 留言的留言
def expand(url):
    driver.get(url)
    driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
    try:
        driver.find_element_by_xpath('//div[@class="_5pcr userContentWrapper"]//a[@data-testid="UFI2CommentsCount/root"]').click()
        time.sleep(1)
        driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
        time.sleep(1)
        driver.find_element_by_id('expanding_cta_close_button').click() 
    except:
        time.sleep(1)
    # 偵測是否有「更多留言」（第一層），若有則點擊
    while len(driver.find_elements_by_xpath('//a[@data-testid="UFI2CommentsPagerRenderer/pager_depth_0"]'))>0:
        for i in driver.find_elements_by_xpath('//a[@data-testid="UFI2CommentsPagerRenderer/pager_depth_0"]'):
            driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
            # 若彈出註冊視窗點擊not now!
            try:
                driver.find_element_by_id('expanding_cta_close_button').click()
            except:
                time.sleep(0.5)            
            try:
                i.click()
            except:
                time.sleep(0.5)   
    # 偵測是否有「更多留言的留言」（第二層），若有則點擊
    while len(driver.find_elements_by_xpath('//a[@data-testid="UFI2CommentsPagerRenderer/pager_depth_1"]'))>0:
        driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
        for i in driver.find_elements_by_xpath('//a[@data-testid="UFI2CommentsPagerRenderer/pager_depth_1"]'):
            driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
            # 若彈出註冊視窗點擊not now!
            try:
                driver.find_element_by_id('expanding_cta_close_button').click()
            except:
                time.sleep(0.5)
            try:
                i.click()
            except:
                time.sleep(0.5)
    # 偵測是否有「更多留言的留言」（第二層），若有則點擊
    while len(driver.find_elements_by_xpath('//a[@class="_5v47 fss"]'))>0:
        for i in driver.find_elements_by_xpath('//a[@class="_5v47 fss"]'):
            driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
            # 若彈出註冊視窗點擊not now!
            try:
                i.click()
            except:
                time.sleep(0.5)

透過逐層搜索的方式，逐步定位我們要找的資訊
在這個環節需要反覆透過Chrome的功能比對資料，需要花一些心力進行比對
另外在這部分也使用了大量個try-except，原因是許多資料是有內容才會出現。例如並非每天貼人都會收到「哈哈」、「生氣」的心情。

In [4]:
# 文章內容與互動摘要
def CrawlPost(soup):
    time.sleep(1)
    # po文區塊
    userContent = soup.find('div', {'class':'_5pcr userContentWrapper'})
    # po文人資訊區塊
    PosterInfo = userContent.find('div', {'class':'l_c3pyo2v0u i_c3pynyi2f clearfix'})
    # 互動摘要區(讚、留言與分享)
    feedback = soup.find('form', {'class':'commentable_item collapsed_comments'})
    # 名稱
    Name = PosterInfo.find('img').attrs['aria-label']
    # ID
    ID = PosterInfo.find('a', {'class':'_5pb8 o_c3pynyi2g _8o _8s lfloat _ohe'}).attrs['href'].split('/?',2)[0].split('/',-1)[-1]
    # 網址
    Link = driver.current_url
    # 發文時間
    try:
        Time = PosterInfo.find('abbr').attrs['title']
    except:
        Time = PosterInfo.find('div', {'class':'_1atc fsm fwn fcg'}).text
    # 文章內容
    try:
        Content = userContent.find('div', {'class':'_5pbx userContent _3576'}).text
    except:
        Content = ""
    # Like
    try:
        Like = feedback.find('span', {'data-testid':'UFI2TopReactions/tooltip_LIKE'}).find('a').attrs['aria-label'].split(' ',2)[0]
    except:
        Like = '0' 
    # Love
    try:
        Love = feedback.find('span', {'data-testid':'UFI2TopReactions/tooltip_LOVE'}).find('a').attrs['aria-label'].split(' ',2)[0]
    except:
        Love = '0' 
    # Haha
    try:
        Haha = feedback.find('span', {'data-testid':'UFI2TopReactions/tooltip_HAHA'}).find('a').attrs['aria-label'].split(' ',2)[0]
    except:
        Haha = '0' 
    # Wow
    try:
        Wow = feedback.find('span', {'data-testid':'UFI2TopReactions/tooltip_WOW'}).find('a').attrs['aria-label'].split(' ',2)[0]
    except:
        Wow = '0' 
    # Sad
    try:
        Sad = feedback.find('span', {'data-testid':'UFI2TopReactions/tooltip_SORRY'}).find('a').attrs['aria-label'].split(' ',2)[0]
    except:
        Sad = '0' 
    # Angry
    try:
        Angry = feedback.find('span', {'data-testid':'UFI2TopReactions/tooltip_ANGER'}).find('a').attrs['aria-label'].split(' ',2)[0]
    except:
        Angry = '0'
    # 留言
    try:
        Comments = feedback.find('a', {'data-testid':'UFI2CommentsCount/root'}).text.split(' ',2)[0]
    except:
        Comments = '0' 
    # 分享
    try:
        Share = feedback.find('span', {'class':'_355t _4vn2'}).text.split(' ',2)[0]
    except:
        Share = '0' 
    Updatetime = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    return pd.DataFrame(
        data = [{'Name':Name,
                 'ID':ID,
                 'Link':Link,
                 'Time':Time,
                 'Content':Content,
                 'Like':Like,
                 'Love':Love,
                 'Haha':Haha,
                 'Wow':Wow,
                 'Sad':Sad,
                 'Angry':Angry,
                 'Comments':Comments,
                 'Share':Share,
                 'Updatetime':Updatetime}],
        columns = ['Name', 'ID', 'Time', 'Content', 'Like', 'Love', 'Haha', 'Wow', 'Sad', 'Angry', 'Comments', 'Share', 'Link', 'Updatetime'])

這邊要留意雖然都是粉絲留言，但實際上分成「回應貼文的留言」與「回應留言的留言」。
函數中的第一個迴圈是用來抓「回應貼文的留言」，第二個則是抓「回應留言的留言」。讀者可以自行比較一下兩個迴圈中不同的地方。

In [5]:
# 留言
def CrawlComment(soup):
    time.sleep(1)
    Comments = pd.DataFrame()
    # po文區塊
    userContent = soup.find('div', {'class':'_5pcr userContentWrapper'})
    # 用戶留言區
    userContent = soup.find('div', {'class':'_5pcr userContentWrapper'})
    # 回應貼文的留言
    for i in userContent.findAll('div', {'data-testid':'UFI2Comment/root_depth_0'}):
        try:
            Content = i.find('span', {'dir':'ltr'}).text
        except:
            Content = 'sticker'
        Comment = pd.DataFrame(data = [{'ID':i.find('a', {'class':' _3mf5 _3mg0'}).attrs['data-hovercard'].split('id=',2)[1],
                                 'Name':i.find('img').attrs['alt'],
                                 'Time':i.find('abbr',{'class':'livetimestamp'}).attrs['data-tooltip-content'],
                                 'Content':Content,
                                 'Link':driver.current_url}],
                        columns = ['ID', 'Name', 'Time', 'Content', 'Link'])
        Comments = pd.concat([Comments, Comment], ignore_index=True)
    
    # 回應留言的留言
    for i in userContent.findAll('div', {'data-testid':'UFI2Comment/root_depth_1'}):
        try:
            Content = i.find('span', {'dir':'ltr'}).text
        except:
            Content = 'sticker'
        Comment = pd.DataFrame(data = [{'ID':i.find('a', {'class':' _3mf5 _3mg1'}).attrs['data-hovercard'].split('id=',2)[1],
                                 'Name':i.find('img').attrs['alt'],
                                 'Time':i.find('abbr',{'class':'livetimestamp'}).attrs['data-tooltip-content'],
                                 'Content':Content,
                                 'Link':driver.current_url}],
                        columns = ['ID', 'Name', 'Time', 'Content', 'Link'])
        Comments = pd.concat([Comments, Comment], ignore_index=True)
    Comments['Updatetime'] = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    return Comments

# 組合應用
組合以上函數，針對逐一爬取List中的貼文，依據貼文的類型做後續處理

- 新貼文：爬取貼文與留言
- 舊貼文但有新留言：更新貼文摘要並爬取留言
- 舊貼文且無新留言：pass
- 其他：查看情況

In [12]:
url = 'https://www.facebook.com/tstartel/'
Links = FindLinks(url = url, n = 15)
# 抓下來所有留言
for i in Links:
    print('Dealing with: ' + i)
    try:
        driver.get(i)
        time.sleep(1)
        soup = BeautifulSoup(driver.page_source)
        # 檢視文章摘要，分成全新文章、舊文章但有新留言、舊文章也無新留言
        nPost = CrawlPost(soup)
        compare = pd.merge(left = nPost.loc[:,['Link', 'Comments']] ,
                           right = Posts.loc[:,['Link', 'Comments']],
                           how = 'inner',
                           on = ['Link'],
                           indicator=True,
                           suffixes=('_n', '_o'))
        # 新貼文直接爬留言
        if len(compare) == 0:
            print('Type: ' + 'New Post')
            expand(i)
            soup = BeautifulSoup(driver.page_source)
            nComments = CrawlComment(soup)
        # 舊文章但有新留言，僅保留新的留言(drop掉已經在資料庫的資料)
        elif compare.Comments_n.values > compare.Comments_o.values:
            # 開始爬留言並將文章摘要匯入
            print('Type: ' + 'Old Post with New Comments')
            expand(i)
            soup = BeautifulSoup(driver.page_source)
            nComments = CrawlComment(soup)
            nComments = pd.merge(left = nComments,
                                 right = Comments,
                                 how = 'left',
                                 on = ['ID', 'Name', 'Content', 'Link'],
                                 suffixes=('', '_o'),
                                 indicator=True)
            nComments = nComments.loc[nComments._merge == 'left_only',:]
            nComments = nComments.drop(['Time_o', 'Updatetime_o', '_merge'], axis=1)
        # 舊文章也無新留言，pass過去
        elif compare.Comments_n.values == compare.Comments_o.values:
            print('Type: ' + 'Old Post with Old Comments')
            nComments = pd.DataFrame()
        # 其他狀況，請釐清問題
        else:
            print('Type: ' + 'Uncertain condiction! Plz Check this website:' + '...')
            nComments = pd.DataFrame()        
        
        # post的資料直接concat在最下面，並duplicate掉，藉以更新互動摘要
        Posts = pd.concat([Posts, nPost], ignore_index=True)
        Posts = Posts.sort_values(by = 'Updatetime', ascending = False)
        Posts = Posts.drop_duplicates(subset ="Link",
                                      keep= 'first',
                                      inplace = False) 
        Comments = pd.concat([Comments, nComments], ignore_index=True, sort=False)
        print('Result: Succed!')
        print('Time Log: ' + datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
        print('----------------------------------------------------------')
        print('\n')
    except:
        print('Result: Failed!')
        print('Time Log: ' + datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
        print('----------------------------------------------------------')
        print('\n')
Posts.to_csv('C:/Users/TL_Yu/Desktop/Posts.csv',index = False)
Comments.to_csv('C:/Users/TL_Yu/Desktop/Comments.csv',index = False)

Dealing with: https://www.facebook.com/tstartel/videos/321955001812559/
Type: New Post
Result: Succed!
Time Log: 2019-05-15 22:14:47
----------------------------------------------------------


Dealing with: https://www.facebook.com/tstartel/posts/3269351643090860
Type: Uncertain condiction! Plz Check this website:...
Result: Succed!
Time Log: 2019-05-15 22:14:51
----------------------------------------------------------


Dealing with: https://www.facebook.com/tstartel/posts/3267141889978502
Type: Old Post with New Comments
Result: Succed!
Time Log: 2019-05-15 22:15:17
----------------------------------------------------------


Dealing with: https://www.facebook.com/tstartel/photos/a.413576055335114/3259789660713725/
Type: Old Post with New Comments
Result: Succed!
Time Log: 2019-05-15 22:15:32
----------------------------------------------------------


Dealing with: https://www.facebook.com/tstartel/photos/a.413576055335114/3264107533615271/
Type: Old Post with Old Comments
Result:

Type: Old Post with Old Comments
Result: Succed!
Time Log: 2019-05-15 22:18:50
----------------------------------------------------------


Dealing with: https://www.facebook.com/tstartel/photos/a.854369967922385/3197740693585289/
Type: Old Post with Old Comments
Result: Succed!
Time Log: 2019-05-15 22:18:53
----------------------------------------------------------


Dealing with: https://www.facebook.com/tstartel/posts/3194840400541985
Type: Old Post with Old Comments
Result: Succed!
Time Log: 2019-05-15 22:18:57
----------------------------------------------------------


Dealing with: https://www.facebook.com/tstartel/photos/a.2482674201758612/3192347447457947/
Type: Old Post with Old Comments
Result: Succed!
Time Log: 2019-05-15 22:19:01
----------------------------------------------------------


Dealing with: https://www.facebook.com/tstartel/videos/1285036481654086/
Type: Old Post with Old Comments
Result: Succed!
Time Log: 2019-05-15 22:19:05
----------------------------------

# Crawl2.0

In [1]:
import pandas as pd
import re, time, requests, datetime, gc
from selenium import webdriver
from bs4 import BeautifulSoup

# 截取塗鴉墻貼文資訊並比對先前資料
如果每次都把所有貼文抓下來，無疑會浪費程式效能，在這裡我將載入先前的資料，並將各篇貼文分成以下三類：

- **新貼文**：截取貼文內容與留言
- **舊貼文但有新留言**：截取所有留言但只將新留言加入資料庫
- **舊貼文也無新留言**：不處理

可想而知，如果設定排程每天執行，大多數資料都會是「舊貼文也無新留言」的類型，加入這個判斷機制將大幅節省系統效能

In [2]:
# 截取塗鴉墻上貼文的留言數
def GetWall_PostCommentCounts(i):
    try:
        CommentCounts = i.find('a', {'data-testid':'UFI2CommentsCount/root'}).text.split(' ',2)[0]
        if 'K' in CommentCounts:
            CommentCounts = int(float(CommentCounts.split('K')[0])*1000)
        else:
            CommentCounts = int(CommentCounts)
    except:
        CommentCounts = 0
    return CommentCounts

# 截取塗鴉墻上貼文的連結
def GetWall_PostLink(i):
    Link = 'https://www.facebook.com' + i.find('a',{'class':'_5pcq'}).attrs['href'].split('?',2)[0]
    return Link

# 截取塗鴉墻上貼文的發佈時間
def GetWall_PostTime(i):
    try:
        Time = i.find('abbr').attrs['title']
        Time = datetime.datetime.strptime(Time, '%m/%d/%y, %I:%M %p')
        Time = Time.strftime("%Y-%m-%d %H:%M")
    except:
        Time = 'Not Post'
    return Time

def CarwlList(urls, n, Posts):
    CheckList = pd.DataFrame(columns = ['Link','Time','CommentCounts']) 
    for i in urls:
        driver.get(i)
        for i in range(n):
            try:
                driver.find_element_by_css_selector('a.pam.uiBoxLightblue.uiMorePagerPrimary').click() # 加載更多貼文的按鈕
            except:
                time.sleep(2)
            # 這裡會跳出要我們登入的大畫面，找到「稍後再說」的按鈕並點擊
            try:
                driver.find_element_by_xpath('//a[@id="expanding_cta_close_button"]').click()
            except:
                time.sleep(0.5)
            soup = BeautifulSoup(driver.page_source)
            for i in soup.find_all('div', {'class':'_5pcr userContentWrapper'}):
                CheckList = pd.concat([CheckList, pd.DataFrame(data = [{'Link':GetWall_PostLink(i),
                                                                        'Time':GetWall_PostTime(i),
                                                                        'CommentCounts':GetWall_PostCommentCounts(i)}],
                                                                 columns = ['Link','Time','CommentCounts'])],
                                       ignore_index = True)
    
    CarwlList = pd.merge(left = CheckList,
                         right = Posts.loc[:,['Link','CommentCounts']],
                         how='left',
                         on='Link',
                         suffixes=('_c', '_p'),
                         indicator=True)
    CarwlList = pd.concat([CarwlList.loc[CarwlList._merge == 'left_only', :], # 新貼文
                           CarwlList.loc[CarwlList.CommentCounts_c > CarwlList.CommentCounts_p, :]], # 舊貼文但有新留言
                          ignore_index=True)
    CarwlList = CarwlList.drop_duplicates(subset = 'Link',
                                          keep= 'first',
                                          inplace = False)
    return CarwlList

# 爬取特定貼文的內容與留言
## 展開貼文與留言

In [3]:
# 點擊不要現在註冊帳號
def clickNotNow():
    try:
        driver.find_element_by_xpath('//a[@id="expanding_cta_close_button"]').click()
    except:
        time.sleep(1)

# 將顯示的留言從系統推薦改成所有留言

def ClosePopup():
    time.sleep(1)
    driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
    clickNotNow()
    time.sleep(0.5)
    clickNotNow()
    driver.find_element_by_xpath('//div[@class="_5pcr userContentWrapper"]//a[@data-testid="UFI2CommentsCount/root"]').click()
    time.sleep(0.5)
    driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
    clickNotNow()
    # 留言排序選new才能抓到全部留言，
    
    # driver.find_element_by_xpath('//a[@data-testid="UFI2ViewOptionsSelector/link"]').click()
    # time.sleep(0.5)
    # try:
    #     driver.find_element_by_partial_link_text('Comments shown in chronological order with the oldest comments')
    # except:
    #     try:
    #         driver.find_element_by_partial_link_text('New comments and those with new replies appear')
    #     except:
    #         print('Plz, Checkthe post arragne type!')

# 打開各篇貼文並展開所有留言 與 留言的留言

def PostExpand():
    time.sleep(2)
    # 偵測是否有「更多留言」（第一層），若有則點擊
    while len(driver.find_elements_by_xpath('//a[@data-testid="UFI2CommentsPagerRenderer/pager_depth_0"]'))>0:
        for i in driver.find_elements_by_xpath('//a[@data-testid="UFI2CommentsPagerRenderer/pager_depth_0"]'):
            driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
            # 若彈出註冊視窗點擊not now!
            clickNotNow()
            try:
                i.click()
                time.sleep(0.5)
            except:
                time.sleep(0.5)   
    # 偵測是否有「更多留言的留言」（第二層），若有則點擊
    while len(driver.find_elements_by_xpath('//a[@data-testid="UFI2CommentsPagerRenderer/pager_depth_1"]'))>0:
        driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
        for i in driver.find_elements_by_xpath('//a[@data-testid="UFI2CommentsPagerRenderer/pager_depth_1"]'):
            driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
            # 若彈出註冊視窗點擊not now!
            clickNotNow()
            try:
                i.click()
                time.sleep(0.5)
            except:
                time.sleep(0.5)
    # 偵測是否有「更多留言的留言」（第二層），若有則點擊
    while len(driver.find_elements_by_xpath('//a[@class="_5v47 fss"]'))>0:
        for i in driver.find_elements_by_xpath('//a[@class="_5v47 fss"]'):
            driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
            # 若彈出註冊視窗點擊not now!
            try:
                i.click()
                time.sleep(0.5)
            except:
                time.sleep(0.5)

## 爬取貼文內容與留言資訊

In [4]:
# 貼文留言數
def GetPost_PosterName(userContent):
    return userContent.find('img').attrs['aria-label']

# 貼文留言數
def GetPost_PosterID(userContent):
    return userContent.find('div', {'class':'_5pcp _5lel _2jyu _232_'}).attrs['id'].split(';')[0].split('feed_subtitle_')[-1]

# 貼文時間
def GetPost_PostTime(userContent):
    try:
        Time = userContent.find('abbr').attrs['title']
        Time = datetime.datetime.strptime(Time, '%m/%d/%y, %I:%M %p')
        Time = Time.strftime("%Y-%m-%d %H:%M")
    except:
        Time = "Error, Please check this post's condiction!" 
    return Time

# 貼文內容
def GetPost_PostContent(userContent):
    try:
        Content = userContent.find('div', {'class':'_5pbx userContent _3576'}).text
    except:
        Content = "There's No Text Content!"
    return Content

# 貼文留言數
def GetPost_PostCommentCounts(userContent):
    try:
        CommentCounts = userContent.find('a', {'data-testid':'UFI2CommentsCount/root'}).text.split(' ',2)[0]
        if 'K' in CommentCounts:
            CommentCounts = int(float(CommentCounts.split('K')[0])*1000)
        else:
            CommentCounts = int(CommentCounts)
    except:
        CommentCounts = 0 
    return CommentCounts

# 貼文分享數
def GetPost_PostShareCounts(userContent):
    try:
        ShareCounts = userContent.find('span', {'class':'_355t _4vn2'}).text.split(' ',2)[0]
        if 'K' in ShareCounts:
            ShareCounts = int(float(ShareCounts.split('K')[0])*1000)
        else:
            ShareCounts = int(ShareCounts)
    except:
        ShareCounts = 0
    return ShareCounts

# 貼文按讚心情數
def GetPost_PostLikeCounts(userContent):
    try:
        LikeCounts = userContent.find('span', {'data-testid':'UFI2TopReactions/tooltip_LIKE'}).find('a').attrs['aria-label'].split(' ',2)[0]
        if 'K' in LikeCounts:
            LikeCounts = int(float(LikeCounts.split('K')[0])*1000)
        else:
            LikeCounts = int(LikeCounts)
    except:
        LikeCounts = 0
    return LikeCounts

# 貼文愛心心情數
def GetPost_PostLoveCounts(userContent):
    try:
        LoveCounts = userContent.find('span', {'data-testid':'UFI2TopReactions/tooltip_LOVE'}).find('a').attrs['aria-label'].split(' ',2)[0]
        if 'K' in LoveCounts:
            LoveCounts = int(float(LoveCounts.split('K')[0])*1000)
        else:
            LoveCounts = int(LoveCounts)
    except:
        LoveCounts = 0
    return LoveCounts

# 貼文哈哈心情數
def GetPost_PostHahaCounts(userContent):
    try:
        HahaCounts = userContent.find('span', {'data-testid':'UFI2TopReactions/tooltip_HAHA'}).find('a').attrs['aria-label'].split(' ',2)[0]
        if 'K' in HahaCounts:
            HahaCounts = int(float(HahaCounts.split('K')[0])*1000)
        else:
            HahaCounts = int(HahaCounts)
    except:
        HahaCounts = 0
    return HahaCounts

# 貼文Wow心情數
def GetPost_PostWowCounts(userContent):
    try:
        WowCounts = userContent.find('span', {'data-testid':'UFI2TopReactions/tooltip_WOW'}).find('a').attrs['aria-label'].split(' ',2)[0]
        if 'K' in WowCounts:
            WowCounts = int(float(WowCounts.split('K')[0])*1000)
        else:
            WowCounts = int(WowCounts)
    except:
        WowCounts = 0
    return WowCounts

# 貼文Sad心情數
def GetPost_PostSadCounts(userContent):
    try:
        SadCounts = userContent.find('span', {'data-testid':'UFI2TopReactions/tooltip_SORRY'}).find('a').attrs['aria-label'].split(' ',2)[0]
        if 'K' in SadCounts:
            SadCounts = int(float(SadCounts.split('K')[0])*1000)
        else:
            SadCounts = int(SadCounts)
    except:
        SadCounts = 0
    return SadCounts

# 貼文Angry心情數
def GetPost_PostAngryCounts(userContent):
    try:
        AngryCounts = userContent.find('span', {'data-testid':'UFI2TopReactions/tooltip_ANGER'}).find('a').attrs['aria-label'].split(' ',2)[0]
        if 'K' in AngryCounts:
            AngryCounts = int(float(AngryCounts.split('K')[0])*1000)
        else:
            AngryCounts = int(AngryCounts)
    except:
        AngryCounts = 0
    return AngryCounts

# 留言時間
def GetPost_CommentTime(element):
    Time = element.find('abbr',{'class':'livetimestamp'}).attrs['data-tooltip-content']
    Time = datetime.datetime.strptime(Time, '%A, %b %d, %Y at %I:%M %p').strftime("%Y-%m-%d %H:%M")
    return Time

# 留言內容
def GetPost_CommentContent(element):
    try:
        Content = element.find('span', {'dir':'ltr'}).text
    except:
        Content = 'img'
    return Content

# 文章內容與互動摘要
def PostInfo(soup):
    # 貼文區
    userContent = soup.find('div', {'class':'_5pcr userContentWrapper'})
    PostContent = pd.DataFrame(data = [{'Name':GetPost_PosterName(userContent),
                                    'ID':GetPost_PosterID(userContent),
                                    'Time':GetPost_PostTime(userContent),
                                    'Content':GetPost_PostContent(userContent),
                                    'CommentCounts':GetPost_PostCommentCounts(userContent),
                                    'Shares':GetPost_PostShareCounts(userContent),
                                    'Like':GetPost_PostLikeCounts(userContent),
                                    'Love':GetPost_PostLoveCounts(userContent),
                                    'Haha':GetPost_PostHahaCounts(userContent),
                                    'Wow':GetPost_PostWowCounts(userContent),
                                    'Sad':GetPost_PostSadCounts(userContent),
                                    'Angry':GetPost_PostAngryCounts(userContent),
                                    'Updatetime':datetime.datetime.now().strftime("%Y-%m-%d %H:%M"),
                                    'Link':driver.current_url,}],
                            columns = ['Name', 'ID', 'Time', 'Content', 'CommentCounts', 'Shares', 'Like', 'Love', 'Haha', 'Wow', 'Sad', 'Angry', 'Updatetime', 'Link'])
    return PostContent

def PostComments(soup):  
    PostComments = pd.DataFrame()
    userContent = soup.find('div', {'class':'_5pcr userContentWrapper'})
    try:
        for i in userContent.select('ul._7a9a > li'):
            # 先抓留言並放在Comment
            Comment = pd.DataFrame(data=[{'ID':i.find('a', {'class':' _3mf5 _3mg0'}).attrs['data-hovercard'].split('id=',2)[1],
                                          'Name':i.find('img').attrs['alt'],
                                          'Time':datetime.datetime.strptime(i.find('abbr',{'class':'livetimestamp'}).attrs['data-tooltip-content'], '%A, %B %d, %Y at %I:%M %p').strftime("%Y-%m-%d %H:%M"),
                                          'Content':GetPost_CommentContent(i),
                                          'RepID':userContent.find('div', {'class':'_5pcp _5lel _2jyu _232_'}).attrs['id'].split(';')[0].split('feed_subtitle_')[-1],
                                          'RepName':userContent.find('img').attrs['aria-label'],
                                          'Link':driver.current_url}],
                                   columns = ['ID', 'Name', 'Time', 'Content','RepID','RepName','Link'])
            PostComments = pd.concat([PostComments, Comment], ignore_index=True)
            # 留言的留言
            for j in i.findAll('div', {'data-testid':'UFI2Comment/root_depth_1'}):
                Comment = pd.DataFrame(data=[{'ID':j.find('a', {'class':' _3mf5 _3mg1'}).attrs['data-hovercard'].split('id=',2)[1],
                                              'Name':j.find('img').attrs['alt'],
                                              'Time':datetime.datetime.strptime(j.find('abbr',{'class':'livetimestamp'}).attrs['data-tooltip-content'], '%A, %B %d, %Y at %I:%M %p').strftime("%Y-%m-%d %H:%M"),
                                              'Content':GetPost_CommentContent(j),
                                              'RepID':i.find('a', {'class':' _3mf5 _3mg0'}).attrs['data-hovercard'].split('id=',2)[1],
                                              'RepName':i.find('img').attrs['alt'],
                                              'Link':driver.current_url}],
                                       columns = ['ID', 'Name', 'Time', 'Content','RepID','RepName','Link'])
                PostComments = pd.concat([PostComments, Comment], ignore_index=True)
        PostComments['Updatetime'] = datetime.datetime.now().strftime("%Y-%m-%d %H:%M")
    except:
        print('Crawl Comments Failed!')
    return PostComments

# 更新貼文與留言資訊
將最新抓到的貼文併回Post表格，並以ID與Time作為關鍵(key)值，取最新抓取的資料(Updatetime)

In [5]:
def UpdateData(DateFrame_o,DateFrame_n):
    DataFrame = pd.concat([DateFrame_o, DateFrame_n], ignore_index=True)
    DataFrame = DataFrame.sort_values(by = 'Updatetime', ascending = False)
    DataFrame = DataFrame.drop_duplicates(subset = ['ID', 'Time', 'Content'],
                                          keep= 'first',
                                          inplace = False)
    return DataFrame

# 爬資料
## 讀取先前資料
若在桌面偵測不到檔案，就創造新的dataframe

In [6]:
try:
    Posts = pd.read_pickle('C:/Users/TL_Yu/Desktop/Posts.plk')
except:
    Posts = pd.DataFrame(columns=['Name', 'ID', 'Time', 'Content', 'CommentCounts', 'Shares', 'Like', 'Love', 'Haha', 'Wow', 'Sad', 'Angry', 'Updatetime', 'Link'])
try:
    Comments = pd.read_pickle('C:/Users/TL_Yu/Desktop/Comments.plk')
except:
    Comments = pd.DataFrame(columns = ['ID', 'Name', 'Time', 'Content', 'RepID', 'RepName','Link', 'Updatetime'])
    
urls = ['https://www.facebook.com/tstartel/',
        'https://www.facebook.com/chtmobile/',
        'https://www.facebook.com/taiwanmobile/',
        'https://www.facebook.com/fareastone/',
        'https://www.facebook.com/Aptg.tw/']

## 產出待爬清單

In [7]:
driver = webdriver.Chrome()
driver.get('https://www.facebook.com/')
# print(driver.get_cookies())
driver.delete_all_cookies()
driver.add_cookie({'domain': '.facebook.com', 'expiry': 1558752322, 'httpOnly': False, 'name': 'locale', 'path': '/', 'secure': True, 'value': 'en_US'})

In [8]:
CarwlList = CarwlList(urls=urls, n=3, Posts = Posts)
CarwlList

,Link,Time,CommentCounts_c,CommentCounts_p,_merge
0,https://www.facebook.com/tstartel/posts/329295...,2019-05-23 02:15,25,NaN,left_only
3,https://www.facebook.com/chtmobile/posts/27210...,2019-05-23 02:00,1,NaN,left_only
4,https://www.facebook.com/chtmobile/photos/a.26...,2019-05-21 02:00,1,NaN,left_only
5,https://www.facebook.com/chtmobile/posts/27207...,2019-05-20 02:00,0,NaN,left_only
9,https://www.facebook.com/chtmobile/photos/a.26...,2019-05-14 02:42,1,NaN,left_only
14,https://www.facebook.com/notes/%E4%B8%AD%E8%8F...,2019-04-18 22:49,20,NaN,left_only
15,https://www.facebook.com/taiwanmobile/posts/24...,2019-05-23 17:00,3,NaN,left_only
16,https://www.facebook.com/taiwanmobile/photos/a...,2019-05-23 03:00,2,NaN,left_only
17,https://www.facebook.com/taiwanmobile/posts/24...,2019-05-22 17:00,0,NaN,left_only
18,https://www.facebook.com/taiwanmobile/posts/24...,2019-05-21 17:00,0,NaN,left_only


如同上面的說明，當CheckLists「merge」欄位為「left_only」時表示該貼文是新貼文；

而當「merge」欄位為「both」，但「Comments_c」的值大於「Comments_p」時，表示該貼文是舊資料但有新留言。

以下將截取這兩類的貼文作為爬取清單

## 執行爬蟲

In [9]:
for i in CarwlList.Link:
    print('Dealing with: ' + i)
    driver.get(i)
    try:
        ClosePopup()
        PostExpand()
        try:
            soup = BeautifulSoup(driver.page_source)
            nPost = PostInfo(soup)
            nComments = PostComments(soup)
            try:
                Posts = UpdateData(DateFrame_o = Posts, DateFrame_n = nPost)
                Comments = UpdateData(DateFrame_o = Comments,DateFrame_n = nComments)
                print('Update Succed!')
            except:
                print('Update Posts or Comments Failed!')
        except:
            print('Crawl Post or Comments Failed!')
    except:
        print('No, Comments, Plz Check this post!')
    gc.collect()
    print('Time Log: ' + datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S") + '\n\n------------------')

Dealing with: https://www.facebook.com/tstartel/posts/3292950597397631
Update Succed!
Time Log: 2019-05-24 10:37:36

------------------
Dealing with: https://www.facebook.com/chtmobile/posts/2721070394589252


C:\Users\TL_Yu\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  


Update Succed!
Time Log: 2019-05-24 10:37:45

------------------
Dealing with: https://www.facebook.com/chtmobile/photos/a.264830403546609/2722896354406656/
Update Succed!
Time Log: 2019-05-24 10:37:55

------------------
Dealing with: https://www.facebook.com/chtmobile/posts/2720795567950068
No, Comments, Plz Check this post!
Time Log: 2019-05-24 10:38:00

------------------
Dealing with: https://www.facebook.com/chtmobile/photos/a.264830403546609/2715269541836004/
Update Succed!
Time Log: 2019-05-24 10:38:10

------------------
Dealing with: https://www.facebook.com/notes/%E4%B8%AD%E8%8F%AF%E9%9B%BB%E4%BF%A1%E8%A1%8C%E5%8B%95%E5%AD%B8%E5%9C%92/%E5%BF%AB%E4%BE%86%E4%B8%80%E8%B5%B7%E4%BA%AB%E7%94%A8hami%E7%93%9C%E5%B8%B6%E4%BE%86%E5%A5%BD%E5%BA%B7%E4%B8%AD%E7%8D%8E%E5%90%8D%E5%96%AE/2741451985881855/
No, Comments, Plz Check this post!
Time Log: 2019-05-24 10:38:15

------------------
Dealing with: https://www.facebook.com/taiwanmobile/posts/2482360498651043
Update Succed!
Time Log: 201

Update Succed!
Time Log: 2019-05-24 11:01:08

------------------
Dealing with: https://www.facebook.com/Aptg.tw/photos/a.165348830171434/2453369231369371/
Update Succed!
Time Log: 2019-05-24 11:01:41

------------------
Dealing with: https://www.facebook.com/Aptg.tw/videos/639447693147224/
Update Succed!
Time Log: 2019-05-24 11:02:23

------------------
Dealing with: https://www.facebook.com/Aptg.tw/photos/a.165348830171434/2454844047888556/
Update Succed!
Time Log: 2019-05-24 11:02:38

------------------
Dealing with: https://www.facebook.com/Aptg.tw/photos/a.165348830171434/2452941871412107/
Update Succed!
Time Log: 2019-05-24 11:04:17

------------------


In [10]:
Posts = Posts.reset_index(drop=True)
Posts = Posts.loc[:,['Name', 'ID', 'Time', 'Content', 'CommentCounts', 'Shares', 'Like', 'Love', 'Haha', 'Wow', 'Sad', 'Angry', 'Updatetime', 'Link']]
Posts

,Name,ID,Time,Content,CommentCounts,Shares,Like,Love,Haha,Wow,Sad,Angry,Updatetime,Link
0,亞太電信Gt智慧生活,162174107155573,2019-05-14 22:30,從前從前，有位美人沉睡了很久👸🏻 直到被心愛的王子親吻後才清醒~ 然而，醒來的美人卻被巫婆再...,2600,804,5100,85,23,0,0,0,2019-05-24 11:04,https://www.facebook.com/Aptg.tw/photos/a.1653...
1,亞太電信Gt智慧生活,162174107155573,2019-05-15 21:30,【🔥網路門市限定🔥299全速吃到飽】 免預繳‼️短約期‼️不降速‼️ 全民瘋搶 #299方案...,30,6,87,1,0,1,0,0,2019-05-24 11:02,https://www.facebook.com/Aptg.tw/photos/a.1653...
2,亞太電信Gt智慧生活,639447693147224:9:0,2019-05-17 00:00,辦門號❗加價購❗ 完成以下步驟就抽 #GalaxyS10+🎉 #留言再抽全聯禮卷100元 小...,874,675,875,0,0,1,0,0,2019-05-24 11:02,https://www.facebook.com/Aptg.tw/videos/639447...
3,亞太電信Gt智慧生活,162174107155573,2019-05-17 20:00,《三國演義-最強武將傳》亞太電信小額付費代收‼ 最強在此，誰敢來戰😎 史詩級戰爭手遊 《三...,560,443,569,1,0,0,0,0,2019-05-24 11:01,https://www.facebook.com/Aptg.tw/photos/a.1653...
4,亞太電信Gt智慧生活,162174107155573,2019-05-20 20:15,🔥$188 玩速上網吃到飽 最後倒數ING🔥 好評再延長❗本月最殺❗錯過只能狂捶心肝❗ 💪追...,827,829,860,1,0,4,0,0,2019-05-24 11:01,https://www.facebook.com/Aptg.tw/photos/a.1653...
5,亞太電信Gt智慧生活,162174107155573,2019-05-22 20:00,【快閃大特惠～ GOGO❗️JAPAN🇯🇵】 亞太暢玩卡出國上網無極限！暢遊日本趴趴Go！ ...,307,302,329,1,0,0,0,0,2019-05-24 11:00,https://www.facebook.com/Aptg.tw/photos/a.1653...
6,亞太電信Gt智慧生活,162174107155573,2019-05-21 00:00,繳稅完覺得藍壽香菇？ 亞太義氣相挺，幫你省省省 💰http://bit.ly/2LwlDcv...,5,1,44,1,0,0,0,0,2019-05-24 11:00,https://www.facebook.com/Aptg.tw/photos/a.1653...
7,亞太電信Gt智慧生活,162174107155573,2019-05-21 20:00,狂賀❗Gt TV x OVO 合法訊源爽爽看❗ 在外看手機，在家用OVO👀 👍160個頻道任...,7,2,47,1,0,0,1,0,2019-05-24 11:00,https://www.facebook.com/Aptg.tw/photos/a.1653...
8,亞太電信Gt智慧生活,162174107155573,2019-05-09 21:00,恭喜！亞太電信5月好康3連發，得獎的是….. 等等！這次留給你來決定好惹😤 蒐集義氣胸章抽好...,1600,1100,3500,31,0,0,0,10,2019-05-24 10:59,https://www.facebook.com/Aptg.tw/photos/a.1653...
9,遠傳電信,162583067088337,2019-05-18 03:30,唱歌最母湯的尷尬時刻！ #留言刷一波 大學生94要夜唱才叫青春啊！ 但你確定你揪的人OK🤔 ...,28,4,63,0,2,1,0,0,2019-05-24 10:58,https://www.facebook.com/fareastone/posts/2538...


In [11]:
Comments = Comments.reset_index(drop=True)
Comments = Comments.loc[:,['ID', 'Name', 'Time', 'Content', 'RepID', 'RepName','Link', 'Updatetime']]
Comments

,ID,Name,Time,Content,RepID,RepName,Link,Updatetime
0,162174107155573,亞太電信Gt智慧生活,2019-05-16 02:01,沒問題！ 美人等你好久了呢😉快到訊息匣確認美人傳給你的訊息吧,100014431869255,陳韋如,https://www.facebook.com/Aptg.tw/photos/a.1653...,2019-05-24 11:04
1,162174107155573,亞太電信Gt智慧生活,2019-05-16 20:46,沒問題！ 美人等你好久了呢😉快到訊息匣確認美人傳給你的訊息吧,100009007956687,廖金章,https://www.facebook.com/Aptg.tw/photos/a.1653...,2019-05-24 11:04
2,162174107155573,亞太電信Gt智慧生活,2019-05-16 20:48,沒問題！ 美人等你好久了呢😉快到訊息匣確認美人傳給你的訊息吧,1821617063,Sun-Chuan Chou,https://www.facebook.com/Aptg.tw/photos/a.1653...,2019-05-24 11:04
3,100000557124309,蔡龍進,2019-05-16 20:48,我要解救稅美人,162174107155573,亞太電信Gt智慧生活,https://www.facebook.com/Aptg.tw/photos/a.1653...,2019-05-24 11:04
4,1661610620,王米娜,2019-05-16 20:46,我要解救稅美人,162174107155573,亞太電信Gt智慧生活,https://www.facebook.com/Aptg.tw/photos/a.1653...,2019-05-24 11:04
5,100006937840473,Pin Liao,2019-05-16 20:46,『我要解救稅美人』,162174107155573,亞太電信Gt智慧生活,https://www.facebook.com/Aptg.tw/photos/a.1653...,2019-05-24 11:04
6,100009007956687,廖金章,2019-05-16 20:46,『我要解救稅美人』,162174107155573,亞太電信Gt智慧生活,https://www.facebook.com/Aptg.tw/photos/a.1653...,2019-05-24 11:04
7,100034808687199,喬伊伊,2019-05-16 20:45,『我要解救稅美人』,162174107155573,亞太電信Gt智慧生活,https://www.facebook.com/Aptg.tw/photos/a.1653...,2019-05-24 11:04
8,100000291872238,廖慧婷,2019-05-16 21:18,『我要解救稅美人』,162174107155573,亞太電信Gt智慧生活,https://www.facebook.com/Aptg.tw/photos/a.1653...,2019-05-24 11:04
9,100008286881402,志堅 葉,2019-05-16 20:39,『我要解救稅美人』,162174107155573,亞太電信Gt智慧生活,https://www.facebook.com/Aptg.tw/photos/a.1653...,2019-05-24 11:04


In [12]:
Posts.to_pickle('C:/Users/TL_Yu/Desktop/Posts.plk')
Comments.to_pickle('C:/Users/TL_Yu/Desktop/Comments.plk')

In [24]:
Posts.to_csv('C:/Users/TL_Yu/Desktop/Posts.csv')
Comments.to_csv('C:/Users/TL_Yu/Desktop/Comments.csv')

# 維護測試區

In [26]:
try:
    Posts = pd.read_pickle('C:/Users/TL_Yu/Desktop/Posts.plk')
except:
    Posts = pd.DataFrame(columns=['Name', 'ID', 'Time', 'Content', 'CommentCounts', 'Shares', 'Like', 'Love', 'Haha', 'Wow', 'Sad', 'Angry', 'Updatetime', 'Link'])
try:
    Comments = pd.read_pickle('C:/Users/TL_Yu/Desktop/Comments.plk')
except:
    Comments = pd.DataFrame(columns = ['ID', 'Name', 'Time', 'Content', 'RepID', 'RepName','Link', 'Updatetime'])
    
urls = ['https://www.facebook.com/tstartel/',
        'https://www.facebook.com/chtmobile/',
        'https://www.facebook.com/taiwanmobile/',
        'https://www.facebook.com/fareastone/',
        'https://www.facebook.com/Aptg.tw/']

In [27]:
driver = webdriver.Chrome()
driver.get('https://www.facebook.com/')
# print(driver.get_cookies())
driver.delete_all_cookies()
driver.add_cookie({'domain': '.facebook.com', 'expiry': 1558752322, 'httpOnly': False, 'name': 'locale', 'path': '/', 'secure': True, 'value': 'en_US'})

In [124]:
https://www.facebook.com/tstartel/photos/a.854369967922385/3277739692252055/

a
b
c
d
e
f
Expand Post Comments Succed!a


In [9]:
driver.get('https://www.facebook.com/chtmobile/photos/a.264830403546609/2722488054447486/')

In [10]:
ClosePopup()

In [11]:
PostExpand()

In [12]:
soup = BeautifulSoup(driver.page_source)

In [13]:
nPost = PostInfo(soup)
nPost

,Name,ID,Time,Content,CommentCounts,Shares,Like,Love,Haha,Wow,Sad,Angry,Updatetime,Link
0,中華電信行動學園,166459533383697,2019-05-17 02:00,【1…2…3 Cheese！ 怎麼拍都麻吉】 走到哪拍到哪！紀錄最珍貴的日常 #隨時上傳分享...,1,3,32,0,0,0,0,0,2019-05-22 09:32,https://www.facebook.com/chtmobile/photos/a.26...


In [15]:
nComments = PostComments(soup)
nComments

,Updatetime


In [55]:
Posts = UpdateData(DateFrame_o = Posts, DateFrame_n = nPost)
Posts

,Name,ID,Time,Content,CommentCounts,Shares,Like,Love,Haha,Wow,Sad,Angry,Updatetime,Link
2,台灣之星,360044337354953,2019-05-18 21:00,好想去日本🇯🇵看繡球花唷 小編們聽 #U姐編 嘟嘟囔囔一週快煩死了 交換個眼色，作戰計劃開始...,55,13,626,8,2,0,0,0,2019-05-20 23:47,https://www.facebook.com/tstartel/photos/a.413...
0,台灣之星,360044337354953,2019-05-19 02:00,#週1福利日 #每週一1230 #週週登場 有沒有人跟 #00C編 一樣 手機沒電會狂症發作...,13,4,299,3,0,5,0,0,2019-05-20 23:46,https://www.facebook.com/tstartel/photos/a.413...
1,台灣之星,360044337354953,2019-05-19 20:52,反正只要一兩塊錢 買個袋子裝一裝，比較方便啦😆 環保餐具用完還要洗，好麻煩 直接拿竹筷和塑膠...,35,13,79,1,0,0,3,0,2019-05-20 23:45,https://www.facebook.com/tstartel/posts/328473...


In [56]:
Comments = UpdateData(DateFrame_o = Comments,DateFrame_n = nComments)
Comments

,ID,Name,Time,Content,RepID,RepName,Link,Updatetime
48,360044337354953,台灣之星,2019-05-19 15:52,看美美照片一樣心情會hen好,100008036701674,陳國進,https://www.facebook.com/tstartel/photos/a.413...,2019-05-20 23:47
61,360044337354953,台灣之星,2019-05-19 19:40,img,100008036701674,陳國進,https://www.facebook.com/tstartel/photos/a.413...,2019-05-20 23:47
70,100008036701674,陳國進,2019-05-20 17:18,台灣之星 還好我16:00回到家過一下才下雨~~,100008036701674,陳國進,https://www.facebook.com/tstartel/photos/a.413...,2019-05-20 23:47
69,360044337354953,台灣之星,2019-05-20 16:48,有沒有記得帶雨傘呀~,100008036701674,陳國進,https://www.facebook.com/tstartel/photos/a.413...,2019-05-20 23:47
68,100008036701674,陳國進,2019-05-20 16:44,台灣之星 小編，下大雨了,100008036701674,陳國進,https://www.facebook.com/tstartel/photos/a.413...,2019-05-20 23:47
67,360044337354953,台灣之星,2019-05-19 21:08,加油加油(๑˃̵ᴗ˂̵)ﻭ其實上課是很幸福D~~晚安😉😉,100008036701674,陳國進,https://www.facebook.com/tstartel/photos/a.413...,2019-05-20 23:47
66,100008036701674,陳國進,2019-05-19 21:03,台灣之星 明天又要上課，小編晚安~~,100008036701674,陳國進,https://www.facebook.com/tstartel/photos/a.413...,2019-05-20 23:47
65,360044337354953,台灣之星,2019-05-19 20:47,早點洗洗睡明天又是新的一天~(✿◠‿◠),100008036701674,陳國進,https://www.facebook.com/tstartel/photos/a.413...,2019-05-20 23:47
64,100008036701674,陳國進,2019-05-19 20:42,台灣之星 是啊～,100008036701674,陳國進,https://www.facebook.com/tstartel/photos/a.413...,2019-05-20 23:47
63,360044337354953,台灣之星,2019-05-19 20:41,國進到家啦❓,100008036701674,陳國進,https://www.facebook.com/tstartel/photos/a.413...,2019-05-20 23:47


In [16]:
driver.quit()

# 其他

In [18]:
# 原本想要登入的方式獲取連結，但FB會偵測異常行為，暫時先不登入
# 關閉Chrome的「通知」提醒
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("--disable-notifications")

driver = webdriver.Chrome(options=chrome_options)
url = 'https://www.facebook.com/'
driver.get(url)
time.sleep(3)

# 啟動瀏覽器並登入Facebook
username = driver.find_element_by_id('email')
username.send_keys('61034b001@gms.ndhu.edu.tw')
passwd=driver.find_element_by_id('pass')
passwd.send_keys('19920309')
button=driver.find_element_by_id('loginbutton')
button.click()